In [ ]:
import mlflow
import pandas as pd

uri = ... # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)


In [2]:

def generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim"
):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )

        for run in runs:
            note = run.data.params.get("note")
            fusion = run.data.params.get("SAE_fusion_strategy")
            group = run.data.params.get("group_type")

            if fusion != aggregation_function or group != group_type:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dataset, dim, topk)
            model_type = "main" if note == main_note else "compare" if note == note_to_compare else None
            if not model_type:
                continue

            metrics = {
                ("G/mean", model_type): run.data.metrics.get("CommonItemsNDCG20/mean"),
                ("U/mean", model_type): run.data.metrics.get("NDCG20/mean"),
                ("U/min", model_type): run.data.metrics.get("NDCG20/min"),
                ("Pop", model_type): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Combine metrics into a dictionary
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dataset", "Dimensions", "TopK"]

    # Calculate % differences
    result_cols = []
    for metric in sorted(set(k[0] for k in df.columns)):
        main_col = (metric, "main")
        compare_col = (metric, "compare")
        percent_col = (metric, "% change")

        if main_col in df.columns and compare_col in df.columns:
            df[percent_col] = ((df[main_col] - df[compare_col]) / df[compare_col].abs()) * 100
            result_cols.extend([main_col, percent_col])
        elif main_col in df.columns:
            result_cols.append(main_col)

    # Keep only main and percent change columns, sort them by metric
    df = df[result_cols]
    df = df.round(2)
    df = df.sort_index(axis=1, level=0).sort_values(by=["Dataset", "Dimensions", "TopK"])
    

    return df.reset_index()

# Normalized embeddings

Jedna z moznych veci, co muze v modelech nastat je, ze velikost sparse embeddingu muze mit odlisnou distribuci mezi cleny skupiny. jinymi slovy, nekdo muze mit vetsi hodnoty embeddingu nez nekdo jiny. Pri agregaci by to pote znamenalo, ze nektere uzivatele budou vice ovlivnovat vysledny embedding nez jini. To muze byt problem, pokud chceme, aby vysledky byly fair pro vsechny uzivatele.

Pojdme nejdrive prozkoumat, zda takovy jev opravdu nastava. Koukneme se na distribuci sumy hodnot v embeddingu pres sample 5000 uzivatelu. Opet vezmeme stejny priklad jako minule tedy dimenzi 2048 a topk 64. Jak je videt na grafu, histogram sum embeddingu uzivatelu tvori normalni rozdeleni. Vsichni uzivatele tedy nemaji stejne hodnoty a normalizace by mohla pomoct udelat uzivatele fairnejsi.


Nyni se jeste pojdme podivat na graf pokud vypneme normalizaci. Jak je videt, zde uz se nejedna o ciste normalni rozdeleni, ale ocasek u vetsich hodnot je mnohem vyraznejsi. I zde by mohla normalizace pomoci, tak aby nebyli nekteri uzivatele vetsiho embeddingu preferovani oproti ostatnim.

Noramlizaci, kterou chceme pouzit je vzit L2 normu embeddingu a pronasobit ji prumernou hodnotou embeddingu. Tedy normalizace bude vypadat takto:

\begin{equation*}
\text{normalized\_embedding} = \frac{\text{embedding}}{\|\text{embedding}\|_2} \cdot \text{mean}(\text{embedding})
\end{equation*}

Podivejme se jak tedy vypadaji normalizovane doporuceni. Nejdrive pro commen features bez aktivace




## SAE group recommendation performance for **common features** aggregation function and **similar groups**

Comparision of base model and model with normalized embeddings

In [3]:
experiment_ids = ['333391697323445885', '523100174176986081']
table = generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_L2_without_acts_normalized",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim"
)
table.groupby("Dataset").mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_57198/1481799203.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -0.201111  0.568889  0.156667  0.586667   
MovieLens  2389.333333  74.666667  0.175556  0.586667  0.284444  0.460000   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -0.046667  0.793333 -0.266667  0.618889  
MovieLens  0.105556  0.662222  0.084444  0.513333

In [9]:
experiment_ids = ['333391697323445885', '523100174176986081']
table = generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_L2_without_acts_normalized",
    note_to_compare="sizes_L2_without_acts",
    group_type="random"
)
table.groupby("Dataset").mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_43729/1561367391.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -0.247778  0.478889  0.112222  0.647778   
MovieLens  2389.333333  74.666667  0.295556  0.533333  0.315556  0.480000   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -0.028889  0.737778 -0.355556  0.523333  
MovieLens  0.145556  0.618889  0.185556  0.461111

In [10]:
experiment_ids = ['333391697323445885', '523100174176986081']
table =generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_L2_without_acts_normalized",
    note_to_compare="sizes_L2_without_acts",
    group_type="outlier"
)
table.groupby("Dataset").mean()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_43729/3164173164.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean()


Dimensions       TopK    G/mean                 Pop            \
                                   % change      main  % change      main   
Dataset                                                                     
LastFM1k   2389.333333  74.666667 -0.308889  0.397778  0.200000  0.606667   
MovieLens  2389.333333  74.666667  0.507778  0.352222  0.606667  0.360000   

             U/mean               U/min            
           % change      main  % change      main  
Dataset                                            
LastFM1k  -0.003333  0.693333 -0.506667  0.443333  
MovieLens  0.241111  0.521111  0.525556  0.340000

Jak je videt, tato zmena je naprosto minimalni a nedochazi k zadne zmene v doporucenich. Nyni se podivejme na average s aktivaci. 

## SAE group recommendation performance for **average** aggregation function and **similar groups**

Comparision of base model and model with normalized embeddings

In [5]:
experiment_ids = ['333391697323445885', '523100174176986081']
generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="average",
    main_note="sizes_L2_with_acts_normalized",
    note_to_compare="sizes_L2_with_acts",
    group_type="sim"
)

Dataset Dimensions TopK   G/mean            Pop         U/mean        \
                              % change  main % change  main % change  main   
0    LastFM1k       1024   32    -0.34  0.58     0.41  0.58    -0.10  0.80   
1    LastFM1k       1024   64    -0.03  0.58     0.37  0.57    -0.03  0.80   
2    LastFM1k       1024  128    -0.54  0.58     0.39  0.57    -0.12  0.80   
3    LastFM1k       2048   32    -0.57  0.59     0.69  0.58    -0.16  0.80   
4    LastFM1k       2048   64    -0.68  0.59     0.55  0.57    -0.19  0.80   
5    LastFM1k       2048  128    -0.55  0.59     0.52  0.57    -0.16  0.80   
6    LastFM1k       4096   32    -0.70  0.59     0.77  0.58    -0.20  0.81   
7    LastFM1k       4096   64    -0.70  0.59     0.48  0.57    -0.20  0.81   
8    LastFM1k       4096  128    -0.67  0.59     0.36  0.57    -0.20  0.80   
9   MovieLens       1024   32    -0.30  0.64    -0.04  0.50    -0.08  0.70   
10  MovieLens       1024   64    -0.03  0.65     0.55  0.51     0.03  0.70   
11  MovieLens       1024  128     0.02  0.65     1.03  0.51    -0.00  0.70   
12  MovieLens       2048   32    -0.25  0.64     0.14  0.51    -0.08  0.70   
13  MovieLens       2048   64    -0.24  0.64     0.45  0.50    -0.04  0.70   
14  MovieLens       2048  128    -0.03  0.65     0.92  0.50    -0.00  0.70   
15  MovieLens       4096   32    -0.18  0.64    -0.07  0.50    -0.04  0.70   
16  MovieLens       4096   64    -0.22  0.64    -0.18  0.50    -0.06  0.70   
17  MovieLens       4096  128    -0.26  0.65     0.52  0.50    -0.08  0.70   

      U/min        
   % change  main  
0     -0.47  0.62  
1     -0.35  0.63  
2     -0.68  0.63  
3     -0.72  0.63  
4     -0.79  0.63  
5     -0.80  0.63  
6     -0.78  0.64  
7     -0.84  0.64  
8     -0.75  0.63  
9     -0.28  0.55  
10    -0.26  0.56  
11    -0.45  0.56  
12    -0.46  0.55  
13    -0.44  0.55  
14    -0.29  0.56  
15    -0.42  0.55  
16    -0.42  0.55  
17    -0.49  0.55

In [6]:
experiment_ids = ['333391697323445885', '523100174176986081']
generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="average",
    main_note="sizes_L2_with_acts_normalized",
    note_to_compare="sizes_L2_with_acts",
    group_type="random"
)

Dataset Dimensions TopK   G/mean            Pop         U/mean        \
                              % change  main % change  main % change  main   
0    LastFM1k       1024   32    -0.66  0.49     0.46  0.64    -0.15  0.75   
1    LastFM1k       1024   64    -0.51  0.49     0.35  0.63    -0.07  0.75   
2    LastFM1k       1024  128    -0.80  0.49     0.33  0.63    -0.12  0.75   
3    LastFM1k       2048   32    -1.08  0.50     0.83  0.65    -0.24  0.75   
4    LastFM1k       2048   64    -0.67  0.50     0.64  0.64    -0.10  0.75   
5    LastFM1k       2048  128    -1.07  0.50     0.49  0.64    -0.18  0.75   
6    LastFM1k       4096   32    -0.80  0.50     0.75  0.65    -0.14  0.76   
7    LastFM1k       4096   64    -1.07  0.50     0.53  0.64    -0.19  0.76   
8    LastFM1k       4096  128    -0.71  0.50     0.34  0.64    -0.12  0.75   
9   MovieLens       1024   32    -0.13  0.63    -0.13  0.55    -0.06  0.69   
10  MovieLens       1024   64    -0.21  0.63     0.34  0.55     0.00  0.69   
11  MovieLens       1024  128    -0.04  0.63     1.23  0.55     0.01  0.69   
12  MovieLens       2048   32    -0.22  0.63    -0.07  0.55    -0.01  0.69   
13  MovieLens       2048   64    -0.08  0.63     0.25  0.55    -0.00  0.69   
14  MovieLens       2048  128     0.01  0.63     0.94  0.55     0.01  0.69   
15  MovieLens       4096   32    -0.65  0.63    -0.26  0.55    -0.15  0.69   
16  MovieLens       4096   64    -0.59  0.63    -0.30  0.54    -0.20  0.69   
17  MovieLens       4096  128    -0.46  0.63     0.23  0.54    -0.08  0.69   

      U/min        
   % change  main  
0     -0.56  0.54  
1     -0.66  0.53  
2     -1.06  0.53  
3     -1.06  0.54  
4     -0.79  0.54  
5     -1.23  0.54  
6     -0.99  0.54  
7     -1.12  0.55  
8     -1.16  0.54  
9     -0.35  0.53  
10    -0.44  0.54  
11    -0.50  0.54  
12    -0.33  0.53  
13    -0.27  0.54  
14    -0.41  0.54  
15    -0.71  0.53  
16    -0.78  0.54  
17    -0.62  0.54

In [7]:
experiment_ids = ['333391697323445885', '523100174176986081']
generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="average",
    main_note="sizes_L2_with_acts_normalized",
    note_to_compare="sizes_L2_with_acts",
    group_type="outlier"
)

Dataset Dimensions TopK   G/mean            Pop         U/mean        \
                              % change  main % change  main % change  main   
0    LastFM1k       1024   32    -0.34  0.41     0.98  0.60     0.06  0.72   
1    LastFM1k       1024   64    -0.36  0.40     0.86  0.58     0.01  0.71   
2    LastFM1k       1024  128    -0.94  0.40     0.88  0.58     0.01  0.71   
3    LastFM1k       2048   32    -0.92  0.42     1.59  0.60    -0.04  0.72   
4    LastFM1k       2048   64    -0.80  0.41     1.25  0.59    -0.04  0.71   
5    LastFM1k       2048  128    -1.04  0.41     1.02  0.58     0.00  0.71   
6    LastFM1k       4096   32    -0.90  0.42     1.45  0.60    -0.12  0.72   
7    LastFM1k       4096   64    -1.51  0.42     1.00  0.59    -0.20  0.72   
8    LastFM1k       4096  128    -1.14  0.41     0.68  0.59    -0.15  0.71   
9   MovieLens       1024   32    -1.19  0.54    -0.77  0.48    -0.30  0.66   
10  MovieLens       1024   64    -0.41  0.55    -0.06  0.48    -0.08  0.67   
11  MovieLens       1024  128     0.77  0.55     2.35  0.49     0.38  0.67   
12  MovieLens       2048   32    -0.80  0.54    -0.71  0.48    -0.11  0.67   
13  MovieLens       2048   64    -0.46  0.54    -0.10  0.47    -0.05  0.66   
14  MovieLens       2048  128     0.07  0.55     1.50  0.48     0.25  0.67   
15  MovieLens       4096   32    -0.50  0.55    -0.48  0.48     0.00  0.67   
16  MovieLens       4096   64    -1.06  0.54    -0.92  0.47    -0.30  0.66   
17  MovieLens       4096  128    -0.58  0.54    -0.41  0.47    -0.22  0.66   

      U/min        
   % change  main  
0     -1.02  0.45  
1     -0.84  0.43  
2     -1.25  0.43  
3     -1.50  0.45  
4     -1.49  0.44  
5     -1.36  0.44  
6     -0.93  0.45  
7     -1.61  0.45  
8     -1.41  0.44  
9     -0.64  0.48  
10    -0.53  0.49  
11     0.26  0.49  
12    -0.69  0.48  
13    -0.81  0.48  
14    -0.57  0.48  
15    -0.72  0.48  
16    -0.85  0.48  
17    -0.64  0.48